In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import nengo
from nengo.utils.matplotlib import rasterplot
import skspeech

import phd

In [ ]:
%%javascript
if($(IPython.toolbar.selector.concat(' > #kill-run-first')).length == 0){
  IPython.toolbar.add_buttons_group([
    {
      'label'   : 'kill and run-first',
      'icon'    : 'fa fa-angle-double-down',
      'callback': function(){
        IPython.notebook.kernel.restart();
        $(IPython.events).one('kernel_ready.Kernel', function(){
          var idx = IPython.notebook.get_selected_index();
          IPython.notebook.select(0);
          IPython.notebook.execute_cell();
          IPython.notebook.select(idx);
        });
      }
    }
  ], 'kill-run-first');
}

In [ ]:
# Compression via log
t = np.linspace(0.5, 20)
plt.plot(t, np.log(t))

In [ ]:
# Compression via 1/3 power law
t = np.linspace(0.5, 20)
plt.plot(t, t ** (1. / 3.))

In [ ]:
from scipy.fftpack import dct, idct
from phd.networks.cepstra import idct as idct_matrix

def my_idct(x):
    """Adapted from https://unix4lyfe.org/dct-1d/"""
    n = x.size
    k = np.arange(n)
    s = np.ones_like(x)
    s[0] = np.sqrt(0.5)
    out = np.sqrt(2. / n) * np.sum(
        s * x * np.cos(np.pi * (k[:, np.newaxis] + 0.5) * k / n), axis=1)
    return out

x = np.random.rand(64)
size_out = 13
print(idct(x, norm='ortho')[:size_out])
print(my_idct(x)[:size_out])
print(np.dot(idct_matrix(x.size, size_out), x))

In [ ]:
# Synthesize a gesture and run it through the auditory model
skspeech.audio = reload(skspeech.audio)
dt = 0.02
freqs = phd.filters.erbspace(20, 2000, 64)
audio_f = skspeech.audio.Gammatone(freqs)
audio_fargs = {'n_cepstrum': 13,
               'remove_0': True,
               'window_dt': 0.01}
x, y, fs = skspeech.vtl.get_traindata(
    'sprachsynthese.ges', audio_f, dt, audio_fargs, 'sprachsynthese.wav')
print x.shape
print y.shape

In [ ]:
plt.pcolormesh(np.clip(x.T, -1, 1))
plt.colorbar()

In [ ]:
plt.pcolormesh(y.T)
plt.colorbar()

In [ ]:
model = phd.Sermo(execution=False)
periphery = model.recognition.periphery
periphery.fs = fs
periphery.freqs = freqs
periphery.sound_process = phd.processes.WavFile('sprachsynthese.wav')
periphery.auditory_filter = phd.filters.gammatone(periphery.freqs)
cepstra = model.recognition.cepstra
cepstra.size_out = 13
net = model.build()

with net:
    ihc_p = nengo.Probe(net.periphery.ihc, synapse=None)
    an_in_p = nengo.Probe(net.periphery.an.input, synapse=None)
    an_p = nengo.Probe(net.periphery.an.add_neuron_output(), synapse=None)
    c_p = nengo.Probe(net.cepstra.output, synapse=0.01)

In [ ]:
print sim.data[c_p]

In [ ]:
#dt = 1. / periphery.freqs.max()
#print("dt=%.5f" % dt)
sim = nengo.Simulator(net, dt=0.0005)
sim.run(1.4)

plt.figure()
phd.plots.cochleogram(sim.data[ihc_p], sim.trange(), periphery.freqs)
plt.figure()
phd.plots.cochleogram(sim.data[an_in_p], sim.trange(), periphery.freqs)
plt.figure()
rasterplot(sim.trange(), sim.data[an_p])
plt.ylim(0, net.periphery.an.n_neurons * net.periphery.an.n_ensembles)
plt.figure()
plt.pcolormesh(np.clip(sim.data[c_p].T, 0, 1))
plt.colorbar()